# 📊 Exploratory Data Analysis - Raw Dataset (VERSIONE AVANZATA)

Questo notebook esegue analisi esplorativa **completa e avanzata** del dataset raw.

## 🎯 Nuove funzionalità implementate:
- **Analisi multi-target**: AI_Prezzo_Ridistribuito + AI_Prezzo_MQ
- **Correlazioni complete**: Pearson + Spearman tra tutte le variabili numeriche
- **Identificazione automatica target** dalla configurazione
- **Confronto statistico** tra target multipli
- **Analisi correlazioni non-lineari** (Spearman)
- **Export completo** dei risultati in CSV

## 1. Setup e caricamento dati

In [ ]:
# Import delle librerie necessarie
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import yaml
import os
import re
from pathlib import Path
from scipy import stats as scipy_stats
from scipy.stats import chi2_contingency, f_oneway
from sklearn.preprocessing import LabelEncoder
import networkx as nx
from scipy.cluster.hierarchy import dendrogram, linkage
from scipy.spatial.distance import squareform
import itertools
import warnings
warnings.filterwarnings('ignore')

# Configurazione matplotlib
plt.style.use('default')
plt.rcParams['figure.figsize'] = (12, 8)
plt.rcParams['font.size'] = 10

print("✅ Setup completato! Tutte le librerie caricate correttamente.")

In [ ]:
# Setup output directory e caricamento dati
output_dir = Path('eda_outputs')
output_dir.mkdir(parents=True, exist_ok=True)
print(f"📁 Directory output: {output_dir}")

# Carica configurazione
config_path = '../config/config.yaml'
try:
    with open(config_path, 'r', encoding='utf-8') as f:
        config = yaml.safe_load(f)
    print(f"✅ Config caricata da: {config_path}")
except Exception as e:
    print(f"❌ Errore nel caricamento config: {e}")
    raise

# Carica dataset
data_path = '../data/raw/raw.parquet'
try:
    df_raw = pd.read_parquet(data_path)
    print(f"✅ Dataset caricato da: {data_path}")
    print(f"📊 Dimensioni: {df_raw.shape[0]:,} righe × {df_raw.shape[1]} colonne")
    print(f"💾 Memoria utilizzata: {df_raw.memory_usage(deep=True).sum() / 1024**2:.2f} MB")
    
    # Identifica target dalla configurazione
    target_candidates = config['target']['column_candidates']
    print(f"🎯 Target candidati dalla configurazione: {target_candidates}")
except Exception as e:
    print(f"❌ Errore nel caricamento dati: {e}")
    raise

## 2. Analisi Multi-Target Avanzata

In [ ]:
# ANALISI MULTI-TARGET AVANZATA
print("=" * 60)
print("🎯 ANALISI MULTI-TARGET AVANZATA")
print("=" * 60)

print(f"\n🎯 Target candidati dalla configurazione: {target_candidates}")

# Verifica presenza dei target nel dataset
available_targets = [col for col in target_candidates if col in df_raw.columns]
missing_targets = [col for col in target_candidates if col not in df_raw.columns]

print(f"✅ Target disponibili: {available_targets}")
if missing_targets:
    print(f"❌ Target mancanti: {missing_targets}")

if not available_targets:
    print("❌ Nessun target disponibile nel dataset!")
    raise ValueError("No target columns found in dataset")

# Analisi per ogni target disponibile
target_stats_all = {}
target_data_all = {}

for target_col in available_targets:
    print(f"\n{'='*50}")
    print(f"🎯 ANALISI TARGET: {target_col}")
    print(f"{'='*50}")
    
    print(f"\n📊 Info target:")
    print(f"  Tipo: {df_raw[target_col].dtype}")
    print(f"  Valori non-nulli: {df_raw[target_col].count():,}")
    print(f"  Valori mancanti: {df_raw[target_col].isnull().sum():,} ({df_raw[target_col].isnull().mean()*100:.2f}%)")

    target_data = df_raw[target_col].dropna()
    target_data_all[target_col] = target_data
    
    if len(target_data) == 0:
        print(f"⚠️  Nessun dato valido per {target_col}, skip")
        continue
        
    stats = target_data.describe()
    target_stats_all[target_col] = stats

    # Unità di misura appropriate
    unit = "€/mq" if "MQ" in target_col.upper() else "€"
    
    print(f"\n📈 Statistiche descrittive:")
    print(f"  Conteggio: {stats['count']:,.0f}")
    print(f"  Media: {unit}{stats['mean']:,.2f}")
    print(f"  Mediana: {unit}{stats['50%']:,.2f}")
    print(f"  Std Dev: {unit}{stats['std']:,.2f}")
    print(f"  Min: {unit}{stats['min']:,.2f}")
    print(f"  Max: {unit}{stats['max']:,.2f}")
    print(f"  Skewness: {scipy_stats.skew(target_data):.4f}")
    print(f"  Kurtosis: {scipy_stats.kurtosis(target_data):.4f}")

# Confronto tra target (se abbiamo più di uno)
if len(available_targets) > 1:
    print(f"\n{'='*60}")
    print("📊 CONFRONTO TRA TARGET")
    print(f"{'='*60}")
    
    # Correlazione tra target
    target_corr_data = df_raw[available_targets].dropna()
    if len(target_corr_data) > 1:
        target_corr = target_corr_data.corr()
        print(f"\n🔗 Correlazione tra target:")
        display(target_corr.round(4))
        
        # Salva correlazione target
        target_corr.to_csv(output_dir / 'target_correlation_matrix.csv')
        print(f"💾 Correlazione target salvata in {output_dir}/target_correlation_matrix.csv")

# Salva statistiche complete per tutti i target
if target_stats_all:
    all_stats_df = pd.DataFrame({target: stats for target, stats in target_stats_all.items()}).T
    all_stats_df.to_csv(output_dir / 'target_statistics_all.csv')
    print(f"\n💾 Statistiche complete salvate in {output_dir}/target_statistics_all.csv")

# Imposta target principale per le analisi successive
primary_target = available_targets[0] if available_targets else None
print(f"\n🎯 Target principale per analisi successive: {primary_target}")

## 3. Analisi Correlazioni Complete

In [ ]:
# ANALISI CORRELAZIONI COMPLETE
print("=" * 60)
print("🔗 ANALISI CORRELAZIONI COMPLETE")
print("=" * 60)

# Identifica colonne numeriche
numeric_cols = df_raw.select_dtypes(include=[np.number]).columns.tolist()
print(f"\n🔢 Colonne numeriche totali: {len(numeric_cols)}")

# Rimuovi colonne costanti
constant_cols = [col for col in numeric_cols if df_raw[col].nunique() <= 1]
analysis_cols = [c for c in numeric_cols if c not in constant_cols]
print(f"⚠️  Colonne costanti rimosse: {len(constant_cols)}")
print(f"✅ Colonne numeriche per analisi: {len(analysis_cols)}")

# Prepara dataset per correlazioni
corr_data = df_raw[analysis_cols].dropna()
print(f"📊 Campioni validi per correlazioni: {len(corr_data):,}")

if len(corr_data) > 0 and len(analysis_cols) > 1:
    
    # 1. CORRELAZIONI PEARSON (lineari)
    print(f"\n{'='*50}")
    print("📈 CORRELAZIONI PEARSON (Lineari)")
    print(f"{'='*50}")
    
    pearson_corr = corr_data.corr(method='pearson')
    
    # Trova correlazioni forti (escludendo diagonale)
    strong_corr = []
    for i in range(len(pearson_corr.columns)):
        for j in range(i+1, len(pearson_corr.columns)):
            corr_val = pearson_corr.iloc[i, j]
            if abs(corr_val) >= 0.3:  # Soglia per correlazioni significative
                strong_corr.append({
                    'Var1': pearson_corr.columns[i],
                    'Var2': pearson_corr.columns[j],
                    'Pearson_Corr': corr_val,
                    'Abs_Corr': abs(corr_val)
                })
    
    strong_corr_df = pd.DataFrame(strong_corr).sort_values('Abs_Corr', ascending=False)
    print(f"\n🔝 Correlazioni Pearson forti (|r| >= 0.3): {len(strong_corr_df)}")
    if len(strong_corr_df) > 0:
        display(strong_corr_df.head(20))
    
    # 2. CORRELAZIONI SPEARMAN (monotone)
    print(f"\n{'='*50}")
    print("📊 CORRELAZIONI SPEARMAN (Monotone)")
    print(f"{'='*50}")
    
    # Campiona per performance se dataset molto grande
    sample_size = min(2000, len(corr_data))
    if sample_size < len(corr_data):
        corr_sample = corr_data.sample(n=sample_size, random_state=42)
        print(f"🎲 Campionamento per performance: {sample_size:,} righe")
    else:
        corr_sample = corr_data
    
    spearman_corr = corr_sample.corr(method='spearman')
    
    # Trova correlazioni Spearman forti
    strong_spearman = []
    for i in range(len(spearman_corr.columns)):
        for j in range(i+1, len(spearman_corr.columns)):
            corr_val = spearman_corr.iloc[i, j]
            if abs(corr_val) >= 0.3:
                strong_spearman.append({
                    'Var1': spearman_corr.columns[i],
                    'Var2': spearman_corr.columns[j],
                    'Spearman_Corr': corr_val,
                    'Abs_Corr': abs(corr_val)
                })
    
    strong_spearman_df = pd.DataFrame(strong_spearman).sort_values('Abs_Corr', ascending=False)
    print(f"\n🔝 Correlazioni Spearman forti (|ρ| >= 0.3): {len(strong_spearman_df)}")
    if len(strong_spearman_df) > 0:
        display(strong_spearman_df.head(15))
    
    # Salva risultati
    pearson_corr.to_csv(output_dir / 'correlation_matrix_pearson.csv')
    spearman_corr.to_csv(output_dir / 'correlation_matrix_spearman.csv')
    
    if len(strong_corr_df) > 0:
        strong_corr_df.to_csv(output_dir / 'strong_correlations_pearson.csv', index=False)
    if len(strong_spearman_df) > 0:
        strong_spearman_df.to_csv(output_dir / 'strong_correlations_spearman.csv', index=False)
    
    print(f"\n💾 Matrici correlazione salvate in {output_dir}/")
    
else:
    print("❌ Dati insufficienti per analisi correlazioni")

## 4. Riepilogo Finale

In [ ]:
# RIEPILOGO FINALE
print("=" * 80)
print("🎉 RIEPILOGO ANALISI ESPLORATIVA AVANZATA COMPLETATA")
print("=" * 80)

print(f"\n📊 Dataset analizzato:")
print(f"  • Dimensioni: {df_raw.shape[0]:,} righe × {df_raw.shape[1]} colonne")
print(f"  • Memoria: {df_raw.memory_usage(deep=True).sum() / 1024**2:.2f} MB")

# Riepilogo target
if 'available_targets' in locals():
    print(f"\n🎯 Target analizzati:")
    for target in available_targets:
        unit = "€/mq" if "MQ" in target.upper() else "€"
        if target in df_raw.columns:
            valid_count = df_raw[target].count()
            mean_val = df_raw[target].mean()
            print(f"  • {target}: {valid_count:,} valori validi, media {unit}{mean_val:,.0f}")

# Riepilogo analisi correlazioni
print(f"\n🔗 Analisi correlazioni eseguite:")
print(f"  ✅ Correlazioni Pearson complete tra tutte le numeriche")
print(f"  ✅ Correlazioni Spearman per relazioni monotone")
print(f"  ✅ Analisi multi-target avanzata")

# Riepilogo file generati
print(f"\n💾 File generati in {output_dir}/:")\ ntry:
    output_files = list(output_dir.glob('*.csv'))
    print(f"  📊 File di analisi: {len(output_files)}")
    for f in sorted(output_files):
        print(f"    • {f.name}")
except:
    print("  📊 Directory output verrà popolata durante l'esecuzione")

print(f"\n🚀 MIGLIORAMENTI IMPLEMENTATI:")
print(f"  ✅ Analisi multi-target (AI_Prezzo_Ridistribuito + AI_Prezzo_MQ)")
print(f"  ✅ Matrice correlazione completa (non solo con target)")
print(f"  ✅ Correlazioni non-lineari (Spearman)")
print(f"  ✅ Identificazione automatica target dalla configurazione")
print(f"  ✅ Confronto statistico tra target multipli")

print(f"\n🎯 PROSSIMI PASSI CONSIGLIATI:")
print(f"  1. Utilizzare le correlazioni forti per feature engineering")
print(f"  2. Sfruttare entrambi i target per modelli multi-output")
print(f"  3. Analizzare le correlazioni Spearman per relazioni non-lineari")
print(f"  4. Usare i file CSV generati per analisi successive")

print(f"\n✅ Analisi esplorativa avanzata completata con successo!")
print(f"Ora puoi procedere con il preprocessing e training con una comprensione più profonda dei dati.")